# Soft Delete Pattern

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/02_core_patterns/soft_delete/soft_delete_pattern.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/02_core_patterns/soft_delete/soft_delete_pattern.ipynb)

## Business Scenario

In regulated environments, hard deletes remove audit history. You need a consistent soft-delete strategy that preserves lineage and supports compliance.

## Value Proposition

- Maintain audit trails without losing records
- Enable easy recovery after mistaken deletes
- Standardize delete metadata across datasets

---

## Goals

1. Flag deleted records
2. Capture deletion timestamps and reasons
3. Keep historical visibility


## 🚀 Step 1: Create the Standardized Contract

We define the standardized system columns in the materialization section.

In [1]:
from pathlib import Path
import shutil
from lakelogic import DataProcessor

BASE = Path.cwd()
if not (BASE / "data").exists():
    candidate = BASE / "examples" / "02_core_patterns" / "soft_delete"
    if (candidate / "data").exists():
        BASE = candidate

contract_yaml = """
version: 1.0.0
dataset: users_silver

source:
  type: table
  cdc_op_field: "op"
  cdc_delete_values: ["D"]

primary_key: ["user_id"]

materialization:
  strategy: merge
  target_path: "./data/users_silver/"
  format: parquet
  soft_delete_column: "_lakelogic_is_deleted"
  soft_delete_value: true
  soft_delete_time_column: "_lakelogic_deleted_at"
  soft_delete_reason_column: "_lakelogic_delete_reason"
"""

contract_path = BASE / "contract.yaml"
contract_path.write_text(contract_yaml, encoding="utf-8")

print("Standardized contract created.")

def preview_frame(df, title=None, limit=10):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "show"):
        try:
            df.show(limit, truncate=False)
            return
        except Exception:
            pass
    if hasattr(df, "head"):
        try:
            head = df.head(limit)
            print(head)
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            sample = df.limit(limit)
            if hasattr(sample, "to_dicts"):
                for row in sample.to_dicts():
                    print(row)
                return
            print(sample)
            return
        except Exception:
            pass
    try:
        if isinstance(df, list):
            for row in df[:limit]:
                print(row)
            return
    except Exception:
        pass
    print(df)

def select_columns(df, columns):
    if hasattr(df, "select"):
        try:
            return df.select(columns)
        except Exception:
            pass
    try:
        if hasattr(df, "__getitem__"):
            return df[columns]
    except Exception:
        pass
    if isinstance(df, list):
        return [{col: row.get(col) for col in columns} for row in df]
    return df

def load_parquet(path):
    view_contract = {
        "version": "1.0.0",
        "dataset": "users_silver_view",
        "source": {"type": "table", "path": str(path)},
    }
    viewer = DataProcessor(contract=view_contract)
    result = viewer.run_source()
    return result.good


Standardized contract created.


## ▶️ Step 2: Initial Load

Start with active users.

In [2]:
output_dir = BASE / "data" / "users_silver"
if output_dir.exists():
    shutil.rmtree(output_dir)

v1_rows = [
    {"user_id": 1, "name": "Alice", "op": "I"},
    {"user_id": 2, "name": "Bob", "op": "I"},
]

processor = DataProcessor(contract=contract_path)
processor.run(v1_rows, source_path="initial_load", materialize=True)

output_path = BASE / "data" / "users_silver" / "data.parquet"
view_df = load_parquet(output_path)
preview_frame(view_df, "Silver table contents (V1):")


2026-02-15 03:32:25.705 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: users_silver]
2026-02-15 03:32:25.751 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 2, Total (post-transform): 2, Good: 2, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%
2026-02-15 03:32:30.103 | INFO     | lakelogic.core.materialization:materialize_dataframe:1096 - Materialized 2 rows to d:\Github\_SaaS\lakelogic\examples\02_core_patterns\soft_delete\data\users_silver\data.parquet
2026-02-15 03:32:30.116 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: d:\Github\_SaaS\lakelogic\examples\02_core_patterns\soft_delete\data\users_silver\data.parquet via polars
2026-02-15 03:32:31.988 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: users_silver_view]
2026-02-15 03:32:31.988 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 2, Total (post-

Silver table contents (V1):
shape: (2, 3)
┌─────────┬───────┬─────┐
│ user_id ┆ name  ┆ op  │
│ ---     ┆ ---   ┆ --- │
│ i64     ┆ str   ┆ str │
╞═════════╪═══════╪═════╡
│ 1       ┆ Alice ┆ I   │
│ 2       ┆ Bob   ┆ I   │
└─────────┴───────┴─────┘


## 🛑 Step 3: Trigger Soft Delete with Metadata

When Bob is deleted, LakeLogic will now automatically populate the `_at` and `_reason` columns.

In [ ]:
v2_rows = [
    {"user_id": 2, "name": "Bob", "op": "D"},
]

processor.run(v2_rows, source_path="soft_delete", materialize=True)

output_path = BASE / "data" / "users_silver" / "data.parquet"
view_df = load_parquet(output_path)

selected = select_columns(
    view_df,
    ["user_id", "name", "_lakelogic_is_deleted", "_lakelogic_deleted_at", "_lakelogic_delete_reason"],
)
preview_frame(selected, "Final silver table with metadata:")
